In [1]:
# Pickle
import pickle

# utils
import numpy as np
import pandas as pd

import nlp

# TF-IDF
from dojo.nlp import TF_IDF

# Dojo Classifiers
from dojo.linear import LogisticRegression
from dojo.bayes import NaiveBayes
from dojo.tree import ClassificationTree

# Model Selection
from dojo.split import cross_validate, train_test_split

# Metrics
from dojo.metrics.classification import accuracy_score

TF-IDF binary not loaded.


***

# Data Loading into Data Frame

In [2]:
data = pd.read_csv("../../data/sentiment.csv", index_col=0)

In [3]:
data.head()

,text,label
0,da vinci code book just awesom,1
1,wa first clive cussler have ever read even boo...,1
2,like da vinci code lot,1
3,like da vinci code lot,1
4,like da vinci code ultimatli not seem hold it is,1


***

# Natural Language Processing

## Standardizing

In [4]:
data["text"] = nlp.standardize(data["text"])

## Remove Noise

In [5]:
data["text"] = nlp.remove_noise(data["text"])

## Lemmatizing / Stemming

In [6]:
# data["text"] = nlp.lemmatize(data["text"])
data["text"] = nlp.stem(data["text"])

## Save data

In [7]:
# data.to_csv(...+"final.csv")

***

# Exploratory Data Analysis

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10086 entries, 0 to 10085
Data columns (total 2 columns):
text     10086 non-null object
label    10086 non-null int64
dtypes: int64(1), object(1)
memory usage: 236.4+ KB


In [9]:
data.describe()

,label
count,10086.000000
mean,0.544815
std,0.498012
min,0.000000
25%,0.000000
50%,1.000000
75%,1.000000
max,1.000000


In [10]:
data.head()

,text,label
0,da vinci code book awesom,1
1,wa first clive cussler ever read even book lik...,1
2,like da vinci code lot,1
3,like da vinci code lot,1
4,like da vinci code ultimatli not seem hold,1


***

# Text Vectorization

In [11]:
X, y = data["text"], data["label"]

In [12]:
try:
    tfidf = pickle.load(open("./tfidf.b", mode="rb"))
except Exception as e:
    tfidf = TF_IDF().fit(X)

In [13]:
X_tf = tfidf.transform(X)

In [14]:
with open("./tfidf.b", mode="wb") as f:
    pickle.dump(tfidf, f)

***

# Model evaluation

In [25]:
rand_idxs = np.random.choice(list(range(len(y))), 500)

In [26]:
X_train, y_train, X_test, y_test = train_test_split(X_tf[rand_idxs, :], y[rand_idxs])

### Logistic Regression

In [27]:
ln = LogisticRegression().fit(X_train, y_train)
accuracy_score(y_test, ln.predict(X_test))

0.86

In [40]:
with open("./logistic.b", mode="wb") as f:
    pickle.dump(ln, f)

### Naive Bayes

In [35]:
bayes = NaiveBayes().fit(X_train, y_train)
accuracy_score(y_test, bayes.predict(X_test))

0.6133333333333333

### Classification Tree

In [39]:
tree = ClassificationTree().fit(X_train, y_train)
accuracy_score(y_test, tree.predict(X_test))

0.6133333333333333

## Grid search (hyper-parameters fitting)